In [619]:
# import Panda dependency
import pandas as pd

In [667]:
# filepath as strings, read as DFs
sch_file = "resources\schools_complete.csv"
stu_file = "resources\students_complete.csv"
sch_df = pd.read_csv(sch_file)
stu_df = pd.read_csv(stu_file)

In [621]:
# clean student names using list, looping through, and using "replace" to change target string into desired value ("") -- blank in this case
presuffixes = ["Dr. ", "Mr. ","Ms. ", "Mrs. ", "Miss ", " MD", " DDS", " DVM", " PhD"]

for psfix in presuffixes:
    stu_df["student_name"] = stu_df["student_name"].str.replace(psfix, "")

C:\Users\nabil\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  """


In [661]:
# merge both dfs for complete data set get basic metrics
all_data = pd.merge(stu_df, sch_df, on=["school_name", "school_name"])

student_count = all_data["Student ID"].count()
school_count = len(all_data["school_name"].unique())
total_budget = sch_df["budget"].sum()
mean_read = all_data["reading_score"].mean()
mean_math = all_data["math_score"].mean()

#pass or fail into new DFs, do counts, and percentages
pass_math = all_data[all_data["math_score"] > 69]
pass_read = all_data[all_data["reading_score"] > 69]
pass_both = all_data[(all_data["math_score"] > 69) & (all_data["reading_score"] > 69)]

math_pass_count = pass_math["student_name"].count()
read_pass_count = pass_read["student_name"].count()
pass_both_count = pass_both["student_name"].count()

math_perc = math_pass_count / float(student_count) * 100
read_perc = read_pass_count / float(student_count) * 100
both_perc = pass_both_count / float(student_count) * 100

In [662]:
# WRANGLING FOR REQUIREMENT #1

#create district summary df
dist_sum_df = pd.DataFrame(
    [{"Total Schools" : school_count,
    "Total Students" : student_count,
    "Total Budget" : total_budget,
    "Ave Math Score" : mean_math,
    "Ave Read Score" : mean_read,
    "% Pass Math" : math_perc,
    "% Pass Read" : read_perc,
    "% Overall Pass" : both_perc}])

#format each column appropriately
dist_sum_df["Total Students"] = dist_sum_df["Total Students"].map("{:,}".format)
dist_sum_df["Ave Math Score"] = dist_sum_df["Ave Math Score"].map("{:.1f}".format)
dist_sum_df["Ave Read Score"] = dist_sum_df["Ave Read Score"].map("{:.1f}".format)
dist_sum_df["% Pass Math"] = dist_sum_df["% Pass Math"].map("{:.0f}".format)
dist_sum_df["% Pass Read"] = dist_sum_df["% Pass Read"].map("{:.0f}".format)
dist_sum_df["% Overall Pass"] = dist_sum_df["% Overall Pass"].map("{:.0f}".format)
dist_sum_df["Total Budget"] = dist_sum_df["Total Budget"].map("${:,.2f}".format)

#reorder columns appropriately
col_order = ["Total Schools","Total Students", "Total Budget", "Ave Math Score", 
             "Ave Read Score", "% Pass Math", "% Pass Read", "% Overall Pass"]
dist_sum_df = dist_sum_df[col_order]

#print output
#dist_sum_df

In [663]:
# WRANGLING FOR REQUIREMENT 2

#determine school types
per_school_types = sch_df.set_index(["school_name"])["type"]
per_school_types

#use value_counts for categorical counting
stud_per_school = stu_df["school_name"].value_counts()

#create series for basic budget table and do calculation
budget_per_school = school_types = sch_df.set_index(["school_name"])["budget"]
per_school_capita = budget_per_school/stud_per_school

#grab mean math and read per school
school_math_mean = all_data.groupby(["school_name"]).mean()["math_score"]
school_read_mean = all_data.groupby(["school_name"]).mean()["reading_score"]

#per school pass math/read create DF and count per DF 
school_pass_math = all_data[(all_data["math_score"] > 69)]
school_pass_read = all_data[(all_data["reading_score"] > 69)]
school_pass_overall = all_data[(all_data["math_score"] > 69) & (all_data["reading_score"] > 69)]
school_pass_m_count = school_pass_math.groupby(["school_name"]).count()["student_name"]
school_pass_r_count = school_pass_read.groupby(["school_name"]).count()["student_name"]
school_pass_all_count = school_pass_overall.groupby(["school_name"]).count()["student_name"]

#calculate percent pass per class per school
math_pass_perc_school = school_pass_m_count / stud_per_school * 100
read_pass_perc_school = school_pass_r_count / stud_per_school * 100
overall_pass_perc_school = school_pass_all_count / stud_per_school * 100

#create  and format output DF
school_sum_df = pd.DataFrame({
    "School Type" : per_school_types,
    "Total Students" : stud_per_school,
    "Total School Budget" : budget_per_school,
    "Per Student Budget" : per_school_capita,
    "Ave Math Score" : school_math_mean,
    "Ave Read Score" : school_read_mean,
    "% Pass Math" : math_pass_perc_school,
    "% Pass Read" : read_pass_perc_school,
    "% Overall Pass" : overall_pass_perc_school})


#FORMAT -- remove formatting if you want to use for calc latr
# school_sum_df["Total Students"] = school_sum_df["Total Students"].map("{:,}".format)
# school_sum_df["Ave Math Score"] = school_sum_df["Ave Math Score"].map("{:.1f}".format)
# school_sum_df["Ave Read Score"] = school_sum_df["Ave Read Score"].map("{:.1f}".format)
# school_sum_df["% Pass Math"] = school_sum_df["% Pass Math"].map("{:.2f}".format)
# school_sum_df["% Pass Read"] = school_sum_df["% Pass Read"].map("{:.2f}".format)
# school_sum_df["% Overall Pass"] = school_sum_df["% Overall Pass"].map("{:.2f}".format)
# school_sum_df["Total School Budget"] = school_sum_df["Total School Budget"].map("${:,.2f}".format)
# school_sum_df["Per Student Budget"] = school_sum_df["Per Student Budget"].map("${:,.2f}".format)

#print output
#school_sum_df

In [664]:
#REQUIREMENT 3.1 Top 5 schools per overall passing

top_sch = school_sum_df.sort_values(["% Overall Pass"], ascending = False)

#top_sch.head(5)

In [665]:
#REQUIREMENT 3.2 Bottom 5 schools per overall passing

bottom_sch = school_sum_df.sort_values(["% Overall Pass"], ascending = True)

#bottom_sch.head(5)

In [666]:
#REQUIREMENT 3.3 Per School, per Grade, Math and READ Ave Scores

grade9 = all_data[(all_data["grade"]== "9th")]
grade10 = all_data[(all_data["grade"] == "10th" )]
grade11 = all_data[(all_data["grade"] == "11th" )]
grade12 = all_data[(all_data["grade"] == "12th" )]

grade9_math = grade9.groupby(["school_name"]).mean()["math_score"]
grade10_math = grade10.groupby(["school_name"]).mean()["math_score"]
grade11_math = grade11.groupby(["school_name"]).mean()["math_score"]
grade12_math = grade12.groupby(["school_name"]).mean()["math_score"]

grade9_read = grade9.groupby(["school_name"]).mean()["reading_score"]
grade10_read = grade10.groupby(["school_name"]).mean()["reading_score"]
grade11_read = grade11.groupby(["school_name"]).mean()["reading_score"]
grade12_read = grade12.groupby(["school_name"]).mean()["reading_score"]

#create and format math per grade per df
math_per_grade = pd.DataFrame ({
    "9th" : grade9_math,
    "10th" : grade10_math,
    "11th" : grade11_math,
    "12th" : grade12_math})

math_per_grade["9th"] = math_per_grade["9th"].map("{:.1f}".format)
math_per_grade["10th"] = math_per_grade["10th"].map("{:.1f}".format)
math_per_grade["11th"] = math_per_grade["11th"].map("{:.1f}".format)
math_per_grade["12th"] = math_per_grade["12th"].map("{:.1f}".format)

#create and format math per grade per school df
read_per_grade = pd.DataFrame ({
    "9th" : grade9_read,
    "10th" : grade10_read,
    "11th" : grade11_read,
    "12th" : grade12_read})

read_per_grade["9th"] = read_per_grade["9th"].map("{:.1f}".format)
read_per_grade["10th"] = read_per_grade["10th"].map("{:.1f}".format)
read_per_grade["11th"] = read_per_grade["11th"].map("{:.1f}".format)
read_per_grade["12th"] = read_per_grade["12th"].map("{:.1f}".format)

# correct order, remove indices if any
math_per_grade = math_per_grade[["9th", "10th", "11th", "12th"]]
read_per_grade = read_per_grade[["9th", "10th", "11th", "12th"]]
math_per_grade.index.name = None
read_per_grade.index.name = None

#print outputs - TWO OUTPUTS
#math_per_grade
#read_per_grade

In [670]:
#Per SPENDING RANGES as indices, raw and ave math score, read score, and overall score per index

#create bins, labels, and CUT data, add to school summary DF
spending_bins = [0,585, 630, 645, 675]
group_names = ["<$584", "$585-629", "$630-644", "$645-675"]
per_school_capita.groupby(pd.cut(per_school_capita, spend_bins)).count()
school_sum_df["Spending Ranges (Per Student)"] = pd.cut(per_school_capita, spending_bins, labels=group_names)

#school summary with spend grouping
spend_math_grade = school_sum_df.groupby(["Spending Ranges (Per Student)"]).mean()["Ave Math Score"]
spend_math_perc = school_sum_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Pass Math"]
spend_read_grade = school_sum_df.groupby(["Spending Ranges (Per Student)"]).mean()["Ave Read Score"]
spend_read_perc = school_sum_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Pass Read"]
spend_overall_perc = school_sum_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Overall Pass"]

#create and format output df
spend_sum_df = pd.DataFrame({
          "Ave Math Score" : spend_math_grade,
          "Ave Read Score": spend_read_grade,
          "% Pass Math": spend_math_perc,
          "% Pass Read": spend_read_perc,
          "% Overall Pass": spend_overall_perc})

spend_sum_df["Ave Math Score"] = spend_sum_df["Ave Math Score"].map("{:.1f}".format)
spend_sum_df["Ave Read Score"] = spend_sum_df["Ave Read Score"].map("{:.1f}".format)
spend_sum_df["% Pass Math"] = spend_sum_df["% Pass Math"].map("{:.0f}".format)
spend_sum_df["% Pass Read"] = spend_sum_df["% Pass Read"].map("{:.0f}".format)
spend_sum_df["% Overall Pass"] = spend_sum_df["% Overall Pass"].map("{:.0f}".format)

#print output
#spend_sum_df

,Ave Math Score,Ave Read Score,% Pass Math,% Pass Read,% Overall Pass
Spending Ranges (Per Student),,,,,
<$584,83.5,83.9,93,97,90
$585-629,81.9,83.2,87,93,81
$630-644,78.5,81.6,73,84,63
$645-675,77.0,81.0,66,81,54


In [679]:
#Per SCHOOL SIZE as indices, aw and ave math score, read score, and overall score per index

#stud_per_school = all_data["school_name"].value_counts()
#stud_per_school

stud_bins = [0, 1000, 2000, 5000]
bin_labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]
school_sum_df["School Size"] = pd.cut(school_sum_df["Total Students"], stud_bins, labels=bin_labels)

size_math_grade = school_sum_df.groupby(["School Size"]).mean()["Ave Math Score"]
size_math_perc = school_sum_df.groupby(["School Size"]).mean()["% Pass Math"]
size_read_grade = school_sum_df.groupby(["School Size"]).mean()["Ave Read Score"]
size_read_perc = school_sum_df.groupby(["School Size"]).mean()["% Pass Read"]
size_overall_perc = school_sum_df.groupby(["School Size"]).mean()["% Overall Pass"]

#create and format output df
size_sum_df = pd.DataFrame({
          "Ave Math Score" : size_math_grade,
          "Ave Read Score": size_read_grade,
          "% Pass Math": size_math_perc,
          "% Pass Read": size_read_perc,
          "% Overall Pass": size_overall_perc})

size_sum_df["Ave Math Score"] = size_sum_df["Ave Math Score"].map("{:.1f}".format)
size_sum_df["Ave Read Score"] = size_sum_df["Ave Read Score"].map("{:.1f}".format)
size_sum_df["% Pass Math"] = size_sum_df["% Pass Math"].map("{:.0f}".format)
size_sum_df["% Pass Read"] = size_sum_df["% Pass Read"].map("{:.0f}".format)
size_sum_df["% Overall Pass"] = size_sum_df["% Overall Pass"].map("{:.0f}".format)

#print output
#size_sum_df

,Ave Math Score,Ave Read Score,% Pass Math,% Pass Read,% Overall Pass
School Size,,,,,
Small (<1000),83.8,83.9,94,96,90
Medium (1000-2000),83.4,83.9,94,97,91
Large (2000-5000),77.7,81.3,70,83,58
